# Introduction

In this project I explore, segment, and cluster the neighborhoods in the city of Toronto. The neighborhood data though is not readily available on the internet.

For the Toronto neighborhood data, a Wikipedia page: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M exists that has all the information we need to explore and cluster the neighborhoods in Toronto. We will have to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format.

Once the data is in a structured format, we can start the analysis to explore and cluster the neighborhoods in the city of Toronto.

We build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood. We only process the cells that have an assigned borough. We ignore cells with a borough that is Not assigned. More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

There are different website scraping libraries and packages in Python. One of the most common packages is BeautifulSoup and we will use it in this project. Package's main documentation page: http://beautiful-soup-4.readthedocs.io/en/latest/

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [7]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
from sklearn.datasets.samples_generator import make_blobs
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
from bs4 import BeautifulSoup
import lxml
print('Libraries imported.')

Libraries imported.


###### Secondly, scrape data set from Wikipedia website
!! We see that, in wikipedia the needs of our data exist as a "table", it is easy to find the table to develop the dataframe with this table data

In [8]:
# download data and parse it:
r = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(r.text, 'html.parser')
table=soup.find('table', attrs={'class':'wikitable sortable'})

table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


###### Thirdly, prepare the dataframe in pursuant of instructions
We need to drop first column and arrange the columns names

In [10]:
# Drop the first column
df.drop(0,inplace=True)
#Rename the columns names
df.columns = ['PostalCode','Borough','Neighborhood']
df.head()

,PostalCode,Borough,Neighborhood
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue



Cleaning data as ignoring the rows which the "Borough" contains 'Not assigned'

In [11]:
# Remove "Borough" with 'Not assigned' values
df2=df[df['Borough'].str.contains("Not assigned") == False].reset_index()
df2.head()

,index,PostalCode,Borough,Neighborhood
0,1,M4A,North York,Victoria Village
1,2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
2,3,M6A,North York,"Lawrence Manor, Lawrence Heights"
3,4,M7A,Queen's Park,Ontario Provincial Government
4,5,M9A,Etobicoke,Islington Avenue


Combine the rows into one row (and seperate with comma) which PostalCode and Borough as same value

In [12]:
df3= df2.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()
df3.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Get the number of rows of dataframe

In [13]:
df3.shape

(102, 3)


we get 103 unique data

Save the file as csv file for using in part 2

In [14]:

#Save data as'Capstone_part1.csv'
df3.to_csv('Capstone_part1.csv')
print('Successfully Saved!')

Successfully Saved!


###### Secondly, get the geospatial Coordinates Data from https://cocl.us/Geospatial_data

In [16]:
# get the data and create the dataframe
url='https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv'
df_geo=pd.read_csv(url)
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Adjust the column name same as the first dataset

In [17]:
df_geo = df_geo.rename(columns = {'Postal Code':'PostalCode'}) 
df_geo.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Upload the first dataset

In [18]:
#We upload the first dataset where we save it as csv file in part 1
df3=pd.read_csv('Capstone_part1.csv')
df3.head()

,Unnamed: 0,PostalCode,Borough,Neighborhood
0,0,M1B,Scarborough,"Malvern, Rouge"
1,1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,3,M1G,Scarborough,Woburn
4,4,M1H,Scarborough,Cedarbrae


Merge two dataframes based on the 'PostalCode' column

In [19]:
df3 = pd.merge(df3, df_geo, on = 'PostalCode')
df3.head()

,Unnamed: 0,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Get the number of rows of dataframe

In [20]:

#Save data as'Capstone_part2.csv'
df3.to_csv('Capstone_part2.csv', index=False)
print('Successfully Saved!')

Successfully Saved!


###### Secondly, Upload the first dataset

In [21]:
#We upload the first dataset where we save it as csv file in part 2
df3=pd.read_csv('Capstone_part2.csv')
df3.head()

,Unnamed: 0,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


###### Thirdly, prepare the dataframe in pursuant of instructions

In [22]:
#We work with only "Borough" which contain the word Toronto 
df4=df3[df3['Borough'].str.contains('Toronto')]
df5=df4.reset_index(drop=True)
df5.head()

,Unnamed: 0,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,36,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,39,M4J,East York/East Toronto,The Danforth East,43.685347,-79.338106
2,40,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
3,41,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
4,42,M4M,East Toronto,Studio District,43.659526,-79.340923


Get the number of rows of dataframe

In [23]:
df5.shape

(39, 6)

we get 38 unique data

For clustering the data, we can use the Borough as Label

In [24]:
# Learn the different value of Borough 
df5['Borough'].value_counts()

Downtown Toronto          17
Central Toronto            9
West Toronto               6
East Toronto               4
Downtown Toronto Stn A     1
East Toronto Business      1
East York/East Toronto     1
Name: Borough, dtype: int64

In [25]:
# Create a new column as Label and get the date from 'Borough' as integer
df5['Label']=df5['Borough'].replace(to_replace=['Downtown Toronto','Central Toronto','West Toronto','East Toronto'],value=[1,2,3,4],inplace=False)
df5.head()

,Unnamed: 0,PostalCode,Borough,Neighborhood,Latitude,Longitude,Label
0,36,M4E,East Toronto,The Beaches,43.676357,-79.293031,4
1,39,M4J,East York/East Toronto,The Danforth East,43.685347,-79.338106,East York/East Toronto
2,40,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,4
3,41,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,4
4,42,M4M,East Toronto,Studio District,43.659526,-79.340923,4


###### Fourthly, visualize the data
Learn the coordinates of Toronto

In [29]:
address = 'Toronto, ON, Canada'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, ON, Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, ON, Canada are 43.6534817, -79.3839347.


Preparing to create the clustering map of Toronto

In [33]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df5['Latitude'], df5['Longitude'], df5['Borough'], df5['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto